In [3]:
import pyodbc
import pandas as pd
import numpy as np
import calendar

import warnings
warnings.filterwarnings("ignore")

In [4]:
conn_str = """
    Driver={ODBC Driver 17 for SQL Server};
    Server=109.71.11.235;
    Database=GFC.Hackathon;
    UID=participant;
    PWD=AiNRYmEatiBL;
"""

conn = pyodbc.connect(conn_str)

In [5]:
# для обучения модели используем записи с IsCorrect is not null
train_query = "select * from dbo.Features f where f.IsCorrect is not null"
train_df = pd.read_sql(train_query, conn).query('ValidationDateTime >= "2023-07-15"') # фильтруем признаки по дате
train_df['ValidationDateTime'] = pd.to_datetime(train_df['ValidationDateTime'])

train_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
10,63,288606,2023-07-21 14:57:04,0.765904,True,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
11,63,288783,2023-07-16 14:56:56,0.456149,True,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
19,63,289136,2023-07-16 14:56:55,0.653798,True,14.0,13.0,0.071429,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
33,63,289599,2023-07-19 14:56:54,0.718939,True,4.0,3.0,0.250000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
35,63,289711,2023-07-20 14:57:04,0.554251,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135814,2950,560531,2023-07-17 15:04:18,0.557407,True,18.0,17.0,0.055556,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135828,2950,562100,2023-07-15 15:02:37,0.758163,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135834,2950,563946,2023-07-19 15:01:09,0.869408,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135842,2950,574768,2023-07-19 15:01:04,0.656481,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0


In [6]:
# для тестирования модели используем записи с IsCorrect is null
test_query = "select * from dbo.Features f where f.IsCorrect is null"
test_df = pd.read_sql(test_query, conn).query('ValidationDateTime >= "2023-07-15"') # фильтруем признаки по дате
test_df['ValidationDateTime'] = pd.to_datetime(test_df['ValidationDateTime'])

test_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288606,2023-07-25 14:57:04,0.406708,None,14.0,13.0,0.071429,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
1,63,288606,2023-08-03 14:58:53,0.460643,None,2.0,1.0,0.500000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
2,63,288783,2023-07-24 14:57:11,0.404959,None,4.0,4.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
3,63,288783,2023-08-03 14:57:39,0.659497,None,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
4,63,289136,2023-07-24 14:57:09,0.299769,None,6.0,6.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,2950,572875,2023-07-26 15:01:26,0.248685,None,19.0,18.0,0.052632,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40767,2950,574768,2023-07-31 15:07:00,0.173554,None,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40768,2950,576631,2023-07-26 15:01:27,0.930926,None,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40769,2950,576631,2023-07-28 15:04:19,0.704421,None,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0


In [7]:
# Убираем колонку IsCorrect для обучения
X_train = train_df.drop("IsCorrect", axis=1)

In [8]:
# Убираем лишнии колонки с большим кол-вом пустых значений
def clear_columns(df):
    df = df.fillna(value=0)
    np.nan_to_num(df)
    cols_to_drop = []
    for column_name in df.columns :
        column = df[column_name]
        percentage = (column == 0).sum() /len(column) * 100
        if percentage > 25 :
            cols_to_drop.append(column_name)

    df = df.drop(cols_to_drop, axis=1)
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    df = df.drop(cols_to_drop, axis=1)
    return df

X_train = clear_columns(X_train)
X_train

,LocationId,ProductId,ValidationDateTime,Probability,Column6,Column7,Column11,Column12,Column14,Column15,...,Column38,Column39,Column40,Column41,Column44,Column53,Column54,Column55,Column56,Column57
10,63,288606,2023-07-21 14:57:04,0.765904,10.0,9.0,3.857143,17,29.990000,29.990000,...,29.990000,29.990000,0.000000,0.000000e+00,0.000000e+00,29.990000,29.990000,29.990000,29.990000,29.990000
11,63,288783,2023-07-16 14:56:56,0.456149,8.0,7.0,2.142857,17,89.989998,89.989998,...,89.989998,99.989998,-10.000000,-1.111235e-01,8.340283e-02,89.989998,89.989998,89.989998,89.989998,89.989998
19,63,289136,2023-07-16 14:56:55,0.653798,14.0,13.0,1.785714,14,24.990000,24.990000,...,24.990000,29.990000,-5.000000,-2.000800e-01,1.667222e-01,24.990000,24.990000,24.990000,24.990000,24.990000
33,63,289599,2023-07-19 14:56:54,0.718939,4.0,3.0,2.321429,14,13.390000,13.389999,...,13.390000,11.421428,1.968573,1.470181e-01,-1.470181e-01,13.390000,13.390000,13.390000,13.390000,13.390000
35,63,289711,2023-07-20 14:57:04,0.554251,0.0,-1.0,0.678571,4,199.899994,199.899994,...,199.899994,0.000000,0.000000,0.000000e+00,0.000000e+00,199.899994,199.899994,199.899994,199.899994,199.899994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135814,2950,560531,2023-07-17 15:04:18,0.557407,18.0,17.0,2.428571,14,11.990000,11.990000,...,11.990000,11.990000,0.000000,0.000000e+00,0.000000e+00,11.990000,11.990000,11.990000,11.990000,11.990000
135828,2950,562100,2023-07-15 15:02:37,0.758163,0.0,-1.0,3.035714,26,69.989998,69.989998,...,69.989998,49.990002,19.999996,2.857551e-01,-2.857551e-01,69.989998,69.989998,69.989998,69.989998,69.989998
135834,2950,563946,2023-07-19 15:01:09,0.869408,0.0,-1.0,2.428571,20,57.990002,57.990002,...,57.990002,62.206665,-4.216663,-7.271363e-02,5.271488e-02,57.990002,57.990002,57.990002,57.990002,57.990002
135842,2950,574768,2023-07-19 15:01:04,0.656481,0.0,-1.0,2.214286,8,19.990002,19.990002,...,19.990002,19.990000,0.000002,9.541513e-08,-7.066872e-08,19.990002,19.990002,19.990002,19.990002,19.990002


In [9]:
# Переводим дату и время в секунды для обработки моделью
X_train['ValidationDateTime'] = X_train['ValidationDateTime'].apply(lambda t : calendar.timegm(t.timetuple())) #дата и время в секунды

In [10]:
# Выбираем таргет для обучения
y_train = train_df["IsCorrect"]
y_train

10        True
11        True
19        True
33        True
35        True
          ... 
135814    True
135828    True
135834    True
135842    True
135846    True
Name: IsCorrect, Length: 24935, dtype: bool

In [11]:
# Данные для тестирования очищаем и приводим в доступный для модели вид  
X_test = clear_columns(test_df)
X_test['ValidationDateTime'] = X_test['ValidationDateTime'].apply(lambda t : calendar.timegm(t.timetuple())) #дата и время в секунды
X_test

,LocationId,ProductId,ValidationDateTime,Probability,Column6,Column7,Column11,Column12,Column14,Column15,...,Column38,Column39,Column40,Column41,Column44,Column53,Column54,Column55,Column56,Column57
0,63,288606,1690297024,0.406708,14.0,13.0,3.071429,15,29.990000,29.990000,...,29.990000,24.990000,5.000000,0.166722,-0.166722,29.990000,29.990000,29.990000,29.990000,29.990000
1,63,288606,1691074733,0.460643,2.0,1.0,1.392857,11,29.990000,29.990000,...,29.990000,29.990000,0.000000,0.000000,0.000000,29.990000,29.990000,29.990000,29.990000,29.990000
2,63,288783,1690210631,0.404959,4.0,4.0,2.000000,16,99.989998,99.989998,...,99.989998,89.989998,10.000000,0.100010,-0.083403,99.989998,99.989998,99.989998,99.989998,99.989998
3,63,288783,1691074659,0.659497,0.0,-1.0,1.964286,18,99.989998,99.989998,...,99.989998,108.900002,-8.910004,-0.089109,0.081818,99.989998,99.989998,99.989998,99.989998,99.989998
4,63,289136,1690210629,0.299769,6.0,6.0,1.357143,13,29.990000,29.990000,...,29.990000,24.990000,5.000000,0.166722,-0.166722,29.990000,29.990000,29.990000,29.990000,29.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,2950,572875,1690383686,0.248685,19.0,18.0,3.785714,25,107.900002,29.287144,...,107.900002,107.900002,0.000000,0.000000,0.000000,107.900002,107.900002,107.900002,107.900002,107.900002
40767,2950,574768,1690816020,0.173554,10.0,9.0,1.321429,4,26.490000,26.490000,...,26.490000,19.990002,6.499998,0.245376,-0.245376,26.490000,26.490000,26.490000,26.490000,26.490000
40768,2950,576631,1690383687,0.930926,1.0,0.0,3.571429,28,139.899994,139.899994,...,139.899994,139.591995,0.307999,0.002202,-0.002202,139.899994,139.899994,139.899994,139.899994,139.899994
40769,2950,576631,1690556659,0.704421,3.0,2.0,3.607143,28,139.684998,139.684998,...,139.684998,139.899994,-0.214996,-0.001539,0.001537,139.684998,139.684998,139.684998,139.684998,139.684998


In [12]:
# Приводим данные для обучения и тестирования к общему количеству колонок
X_test = X_test.drop(X_test.columns.difference(X_train.columns)[0], axis=1)

In [13]:
# Нормализуем данные
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test == ss_test.fit_transform(X_test)

,LocationId,ProductId,ValidationDateTime,Probability,Column6,Column7,Column11,Column12,Column14,Column15,...,Column38,Column39,Column40,Column41,Column44,Column53,Column54,Column55,Column56,Column57
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40767,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40768,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40769,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
# Создаём модель
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [15]:
# Обучаем модель и делаем предстказание
model.fit(X_train,y_train)
predict_proba = model.predict_proba(X_test)
predict_proba

array([[0.59, 0.41],
       [0.59, 0.41],
       [0.53, 0.47],
       ...,
       [0.51, 0.49],
       [0.47, 0.53],
       [0.54, 0.46]])

In [16]:
# Формируем данные для чек-поинта
checkpoint_df = test_df.drop("IsCorrect", axis=1)
checkpoint_df = checkpoint_df.drop(checkpoint_df.iloc[:, checkpoint_df.columns.get_loc('Probability')+1 : len(checkpoint_df.columns)], axis=1)
checkpoint_df['CalculatedProbability'] = predict_proba[:, 1]
checkpoint_df

,LocationId,ProductId,ValidationDateTime,Probability,CalculatedProbability
0,63,288606,2023-07-25 14:57:04,0.406708,0.41
1,63,288606,2023-08-03 14:58:53,0.460643,0.41
2,63,288783,2023-07-24 14:57:11,0.404959,0.47
3,63,288783,2023-08-03 14:57:39,0.659497,0.41
4,63,289136,2023-07-24 14:57:09,0.299769,0.43
...,...,...,...,...,...
40766,2950,572875,2023-07-26 15:01:26,0.248685,0.44
40767,2950,574768,2023-07-31 15:07:00,0.173554,0.45
40768,2950,576631,2023-07-26 15:01:27,0.930926,0.49
40769,2950,576631,2023-07-28 15:04:19,0.704421,0.53


In [17]:
checkpoint_df.to_csv(r'c:\Users\ivan\Desktop\hackaton\BestSales_2', sep=';', index=False)

In [18]:
csv = pd.read_csv(r'c:\Users\ivan\Desktop\hackaton\BestSales_2', sep=';')
csv

,LocationId,ProductId,ValidationDateTime,Probability,CalculatedProbability
0,63,288606,2023-07-25 14:57:04.000,0.406708,0.41
1,63,288606,2023-08-03 14:58:53.000,0.460643,0.41
2,63,288783,2023-07-24 14:57:11.000,0.404959,0.47
3,63,288783,2023-08-03 14:57:39.000,0.659497,0.41
4,63,289136,2023-07-24 14:57:09.000,0.299769,0.43
...,...,...,...,...,...
40766,2950,572875,2023-07-26 15:01:26.000,0.248685,0.44
40767,2950,574768,2023-07-31 15:07:00.000,0.173554,0.45
40768,2950,576631,2023-07-26 15:01:27.000,0.930926,0.49
40769,2950,576631,2023-07-28 15:04:19.000,0.704421,0.53
